In [1]:
import pandas as pd 

In [2]:
siniestros = pd.read_csv('Datasets\clean\siniestos_limpio.csv')

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\maria\AppData\Local\Temp\ipykernel_6592\3795941370.py:1: SyntaxWarning: invalid escape sequence '\c'
  siniestros = pd.read_csv('Datasets\clean\siniestos_limpio.csv')


In [3]:
siniestros

,Id,Fecha,Año,Mes,Día,Día semana,Hora completa,Hora,Num víctimas,Participantes,...,Tipo de calle,Calle,Cruce,Dirección normalizada,Comuna,Xy (caba),Coordenada x,Coordenada y,Pos x,Pos y
0,2016-0001,2016-01-01,2016.0,1.0,1.0,Viernes,04:00:00,4.0,1,MOTO-AUTO,...,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8.0,Point (98896.78238426 93532.43437792),98896.782384,93532.434378,-58.475340,-34.687570
1,2016-0002,2016-01-02,2016.0,1.0,2.0,Sabado,01:15:00,1.0,1,AUTO-PASAJEROS,...,GRAL PAZ,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9.0,Point (95832.05571093 95505.41641999),95832.055711,95505.416420,-58.508775,-34.669777
2,2016-0003,2016-01-03,2016.0,1.0,3.0,Domingo,07:00:00,7.0,1,MOTO-AUTO,...,AVENIDA,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,1.0,Point (106684.29090040 99706.57687843),106684.290900,99706.576878,-58.390403,-34.631894
3,2016-0004,2016-01-10,2016.0,1.0,10.0,Domingo,00:00:00,0.0,1,MOTO-SD,...,AVENIDA,LARRAZABAL AV.,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8.0,Point (99840.65224780 94269.16534422),99840.652248,94269.165344,-58.465039,-34.680930
4,2016-0005,2016-01-21,2016.0,1.0,21.0,Jueves,05:20:00,5.0,1,MOTO-PASAJEROS,...,AVENIDA,SAN JUAN AV.,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1.0,Point (106980.32827929 100752.16915795),106980.328279,100752.169158,-58.387183,-34.622466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0092,2021-12-12,2021.0,12.0,12.0,Domingo,06:20:00,6.0,1,PEATON-AUTO,...,AVENIDA,RIVADAVIA AV.,PUEYRREDON AV.,RIVADAVIA AV. y PUEYRREDON AV.,3.0,Point (105258.35368554 102122.93231400),105258.353686,102122.932314,-58.405969,-34.610120
713,2021-0093,2021-12-13,2021.0,12.0,13.0,Lunes,17:10:00,17.0,1,MOTO-AUTO,...,AVENIDA,RIESTRA AV.,MOM,RIESTRA AV. y MOM,7.0,Point (102728.60090138 98186.24929177),102728.600901,98186.249292,-58.433538,-34.645616
714,2021-0094,2021-12-20,2021.0,12.0,20.0,Lunes,01:10:00,1.0,1,MOTO-AUTO,...,AUTOPISTA,"DELLEPIANE, LUIS, TTE. GRAL.",LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9.0,Point (99624.29795829 97569.69801131),99624.297958,97569.698011,-58.467398,-34.651178
715,2021-0095,2021-12-30,2021.0,12.0,30.0,Jueves,00:43:00,0.0,1,MOTO-CARGAS,...,AVENIDA,GAONA AV.,TERRADA,GAONA AV. y TERRADA,11.0,Point (99116.45492358 101045.23284826),99116.454924,101045.232848,-58.472934,-34.619847
